In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Convert the default LUT to a smoother version.
# The default LUT is symmetric, so we only need one axis.
x16 = np.linspace(0,255,16)
values16_stage = np.array([0, 15, 30, 46, 64, 82, 101, 121, 140, 158, 176, 193, 209, 224, 240, 255])
plt.plot(x16, values16_stage)

In [ ]:
# Define the steps in the post processing as functions.
# TODO: Edit to include gamma correction
def f(x):
    return x * 0.9375 + 0.03125

def lut(x, x16, values16):
    return np.interp(x, x16, values16)

def g(x, srgb):
    return ((x - srgb) * 0.99961 + srgb) * 1.3703

# Apply the original stage LUT.
srgb = np.arange(256)
result = g(lut(f(srgb), x16, values16_stage), srgb)
result = result.clip(0.0, 255.0)

plt.figure()
plt.title('Result')
plt.imshow(np.array([result]*64),cmap='gray',vmin=0,vmax=255.0)

# Apply a new lut to the stage screenshot.
values16_edit = values16_stage * 0.5
result_corrected = lut(result, x16, values16_edit)
result_corrected = result_corrected.clip(0.0, 255.0)

plt.figure()
plt.title('Edited Result')
plt.imshow(np.array([result_corrected]*64),cmap='gray',vmin=0,vmax=255.0)

# Solve for the final stage LUT.
# This should be identical to the edited result above.
values16_final = np.zeros(16)
for i,xi in enumerate(x16):
    # Given xi = f(x), we want to solve for lut_final(xi).
    def f_inv(xi): 
        return (xi - 0.03125) / 0.9375

    x = f_inv(xi)

    def g_x_inv(xi):
        return (((xi / 1.3703) - x) / 0.99961) + x
    print(xi)

    result_stage = g(lut(xi, x16, values16_stage), x)
    result_stage_edited = lut(result_stage, x16, values16_edit)
    values16_final[i] = g_x_inv(result_stage_edited)

result_final = g(lut(f(srgb), x16, values16_final), srgb)
result_final = result_final.clip(0.0, 255.0)

plt.figure()
plt.title('Final Reconstructed Result')
plt.imshow(np.array([result_final]*64),cmap='gray',vmin=0,vmax=255.0)

plt.figure()
plt.title('Absolute Error')
abs_error = np.abs(result_final - result_corrected)
plt.imshow(np.array([abs_error]*64),cmap='gray',vmin=0,vmax=255.0)
plt.figure(figsize=(6,2))
plt.title('Absolute Error')
plt.plot(np.arange(256), abs_error)
print(f'Min Error: {abs_error.min()}, Max Error: {abs_error.max()}')

In [ ]:
x_actual = [0.0, 32/255]
y_actual = [0.0, 0.01599]

plt.plot(x_actual, y_actual)

Suppose we know 
$$result = srgb(g(lut_{stage}(f(x)), x))$$
Given $lut_{edit}(result)$, can we find $lut_{final}$ such that  
$$srgb(g(lut_{final}(f(x)), x)) = lut_{edit}(result)$$

Let $x \in [0,1]$  
We want $$srgb(g(lut_{final}(f(x)), x)) = lut_{edit}(result)$$
Substituting $result$ we get
$$srgb(g(lut_{final}(f(x)), x)) = lut_{edit}(srgb(g(lut_{stage}(f(x)), x)))$$

Define $g_{x}(y) = g(y, x)$ since $g$ itself is not invertible.  
$$srgb(g_x(lut_{final}(f(x)))) = lut_{edit}(srgb(g_x(lut_{stage}(f(x)))))$$
$g_x$ and $f$ are both invertible, and $linear$ and $srgb$ are inverses of each other, so 
$$lut_{final}(f(x)) = g_x^{-1}(linear(lut_{edit}(srgb(g_x(lut_{stage}(f(x)))))))$$

Now we can construct the lookup table $lut_{final}$ by sampling points $x_i = f(x)$.